In [ ]:
# 1. Setup entorno y conexión
import os, sys, json, pandas as pd, geopandas as gpd
from pathlib import Path
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
load_dotenv()
RAW = Path('../data/raw')
PROCESSED = Path('../data/processed')
POSTGRES_USER=os.getenv('POSTGRES_USER'); POSTGRES_PASSWORD=os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST=os.getenv('POSTGRES_HOST','localhost'); POSTGRES_PORT=os.getenv('POSTGRES_PORT','5432')
POSTGRES_DB=os.getenv('POSTGRES_DB')
engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')
print('Conexión lista:', engine)

In [ ]:
# 2. Verificación de fuentes mínimas locales
expected_files = {
 'manzanas_censales.geojson': RAW/'manzanas_censales.geojson',
 'uso_suelo_minvu.geojson': RAW/'uso_suelo_minvu.geojson',
 'comuna_boundaries_oficial.geojson': RAW/'comuna_boundaries_oficial.geojson',
 'Censo2017_Manzanas.csv': RAW/'Censo2017_ManzanaEntidad_CSV'/'Censo2017_16R_ManzanaEntidad_CSV'/'Censo2017_Manzanas.csv'
}
status_rows=[]
for name, path in expected_files.items():
    status_rows.append({'archivo': name, 'existe': path.exists(), 'path': str(path)})
pd.DataFrame(status_rows)

### Nota
Si faltan archivos, ejecute su pipeline de descarga (cuando esté implementado) antes de continuar.

In [ ]:
# 3. Ejecutar ingest mínima (opcional si ya se corrió).
# Para reproducibilidad en notebook usamos subprocess.
import subprocess, shlex
cmd = 'python ../scripts/process_data.py --ingest-minimum --srid 32719'
print('Ejecutando:', cmd)
result = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [ ]:
# 4. Metadatos y cobertura (raw_data)
with engine.begin() as conn:
    orphan_sql = text("SELECT COUNT(*) FROM raw_data.censo_microdatos c LEFT JOIN raw_data.manzanas_censales m ON c.manzent = m.manzent WHERE m.manzent IS NULL")
    micro_sin_geom = conn.execute(orphan_sql).scalar()
    cov_micro = conn.execute(text("SELECT ROUND(100.0 * (SELECT COUNT(*) FROM raw_data.censo_microdatos c JOIN raw_data.manzanas_censales m ON c.manzent=m.manzent)/ NULLIF((SELECT COUNT(*) FROM raw_data.censo_microdatos),0),2)")).scalar()
    cov_manz = conn.execute(text("SELECT ROUND(100.0 * (SELECT COUNT(DISTINCT m.manzent) FROM raw_data.manzanas_censales m JOIN raw_data.censo_microdatos c ON c.manzent=m.manzent)/ NULLIF((SELECT COUNT(*) FROM raw_data.manzanas_censales),0),2)")).scalar()
quality = pd.DataFrame([{'microdatos_sin_geometria': micro_sin_geom,'pct_micro_con_geom': cov_micro,'pct_manz_con_micro': cov_manz}])
quality

In [ ]:
# 5. Verificar alias 'manzent' presencia en tablas clave
tables = [('raw_data','censo_microdatos'), ('raw_data','manzanas_censales')]
rows=[]
with engine.begin() as conn:
    for sch, tbl in tables:
        q = text("SELECT column_name FROM information_schema.columns WHERE table_schema=:s AND table_name=:t AND column_name='manzent'")
        exists = conn.execute(q, {'s': sch, 't': tbl}).fetchone() is not None
        rows.append({'schema': sch,'table': tbl,'has_manzent': exists})
pd.DataFrame(rows)

In [ ]:
# 6. Resumen CRS de capas vectoriales cargadas
vector_tables = ['manzanas_censales','uso_suelo_minvu','comuna_boundaries_oficial']
crs_rows=[]
for tbl in vector_tables:
    try:
        gdf = gpd.read_postgis(f'SELECT * FROM raw_data.{tbl} LIMIT 5', engine, geom_col='geometry')
        crs_rows.append({'tabla': tbl, 'crs': gdf.crs.to_string() if gdf.crs else None})
    except Exception as e:
        crs_rows.append({'tabla': tbl, 'crs': None, 'error': str(e)})
pd.DataFrame(crs_rows)

In [ ]:
# 7. Guardar tabla de calidad a CSV (tracking)
out_csv = PROCESSED/'quality_acquisition_summary.csv'
quality.to_csv(out_csv, index=False)
print('Guardado resumen calidad en', out_csv)

## Checklist de Adquisición
- [x] Conexión PostGIS establecida
- [x] Fuentes mínimas verificadas
- [x] Ingest mínima ejecutada
- [x] Alias `manzent` presente en tablas clave
- [x] Cobertura y huérfanas calculadas
- [x] Metadatos CRS revisados
## Próximos pasos
Continuar con `02_exploratory_analysis.ipynb` para análisis descriptivo y construcción de mapas temáticos iniciales.